# Anális de registro de hospitales y clinicas veterinarias en México.

### Objetivo:

Es un análisis inicial como parte de un proyecto mas grande para establecer un hospital veterinario en méxico, lo que queremos observar es la distribución por el pais de los negocios ya establecidos, asi que conocer que tipo de servicio es el que ofrece, con la finalidad de empezar a definir el nivel de competencia y de formalidad que hay.

In [ ]:
#importar librerias
import pandas as pd
import plotly.express as px


In [2]:
#carga de datos
df= pd.read_csv('C:/Users/alexa/OneDrive/Desktop/clinicas_hospitales_vet.csv')

In [3]:
#Primera observación de los datos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   numero_expediente       2245 non-null   object
 1   empresa                 2245 non-null   object
 2   estado                  2245 non-null   object
 3   municipio_o_delegacion  2245 non-null   object
 4   clinica_veterinaria     2245 non-null   object
 5   farmacia_veterinaria    2245 non-null   object
 6   hospital_veterinario    2245 non-null   object
dtypes: object(7)
memory usage: 122.9+ KB


In [4]:
df.sample(10)

,numero_expediente,empresa,estado,municipio_o_delegacion,clinica_veterinaria,farmacia_veterinaria,hospital_veterinario
510,00467-28-16,Nadia Aracely Ceniceros Pong,Tamaulipas,Madero,si,si,no
126,0027-14-15,Alvaro Garcia Alvarez,Jalisco,Zapopan,si,no,no
1848,1897-16-23,Alejandro Ernesto Herrera Ramirez,Michoacan,Morelia,si,no,si
219,0132-02-15,Jair Enrique Fuentes Castro,Baja California,Mexicali,si,si,no
1769,1813-05-23,Cuauhtemoc Ramirez de Avila,Coahuila,Torreon,si,no,no
1591,1625-01-21,Gerardo Bernal Reza,Aguascalientes,Aguascalientes,si,no,no
1968,24/09/2017,Maria Patricia Garcia Vargas,Ciudad de Mexico,Azcapotzalco,no,no,si
1298,1308-28-19,Luis Daniel Garcia Munoz,Tamaulipas,Reynosa,si,si,si
1097,1095-02-18,Irving Silva Reyes y O Veterinaria Dovet,Baja California,Tijuana,si,no,no
768,00756-16-16,Florencia Ramirez Morales,Michoacan,Morelia,si,si,si


No observo errores en los datos que afecten mi analisis, por lo que no hare ninfuna limpieza.

### Enriquecimiento de los datos.

* categorizar las empresas en persona fisica o persona  moral
* Cambiar columnas de si y no a numericas(0,1) para facilitar mi analisis.


In [5]:
#Se crea y se aplica una función para categorizar el nombre de la empresa en base a las palabras que tiene su nombre

def clasificar_empresa(nombre):
    nombre = nombre.lower()
    if any(razon in nombre for razon in ['s de rl', 's.a.', 's.a. de c.v.', 's. de r.l.', 'sapi', 's.c.', 'cv']):
        return 'persona_moral'
    elif any(palabra in nombre for palabra in ['veterinarios', 'hospital', 'centro', 'servicios', 'clínica', 'grupo', 'corporativo', 'friends', 'pets']):
        return 'persona_moral'
    elif 'y o' in nombre or len(nombre.split()) >= 3:
        return 'persona_fisica'
    else:
        return 'indeterminado'

df['tipo_empresa'] = df['empresa'].apply(clasificar_empresa)

In [6]:
columnas_numericas = ['clinica_veterinaria', 'farmacia_veterinaria', 'hospital_veterinario']

df[columnas_numericas] = (
    df[columnas_numericas]
      .replace({'si': 1, 'no': 0})
      .apply(pd.to_numeric, errors='coerce')  # valores inválidos  NaN
      .astype('Int64') )

# Eliminar filas que tengan NaN en *cualquiera* de esas columnas
df = df.dropna(subset=columnas_numericas)
df[columnas_numericas] = df[columnas_numericas].astype(int)

In [7]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
Index: 2219 entries, 0 to 2244
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   numero_expediente       2219 non-null   object
 1   empresa                 2219 non-null   object
 2   estado                  2219 non-null   object
 3   municipio_o_delegacion  2219 non-null   object
 4   clinica_veterinaria     2219 non-null   int64 
 5   farmacia_veterinaria    2219 non-null   int64 
 6   hospital_veterinario    2219 non-null   int64 
 7   tipo_empresa            2219 non-null   object
dtypes: int64(3), object(5)
memory usage: 156.0+ KB


### Análisis de los datos

* Grafico de dispersión que muestra la frecuencia con la que los dos tipos de empresa tienen determinada cantidad de servicios.
* Creacion de un histograma con la distribucion de acuerdo al estado


In [ ]:

# Calcular cobertura total
df['cobertura_total'] = df[['clinica_veterinaria', 'farmacia_veterinaria', 'hospital_veterinario']].sum(axis=1)


df['tipo_empresa_legible'] = df['tipo_empresa'].map({
    'persona_fisica': 'Persona Física',
    'persona_moral': 'Persona Moral'
})

# Crear gráfico con Plotly Express
agrupado = df.groupby(['tipo_empresa_legible', 'cobertura_total']).size().reset_index(name='frecuencia')

fig = px.scatter(
    agrupado,
    x='tipo_empresa_legible',
    y='cobertura_total',
    size='frecuencia',
    color='frecuencia',
    title='Frecuencia de empresas por tipo y cobertura',
    labels={'tipo_empresa_legible': 'Tipo de Empresa', 'cobertura_total': 'Cobertura Total'},
    template='plotly_white'
)
fig.show()




Podemos observar concluciónes interesantes de este grafico, por ejemplo observamos que el tipo de veterinaria que es mas abundante son, las personas fisicas y que solo ofrecen un tipo de servicio y las que son mas escasas son las personas morales con los tres servicios.
Esto probablemente tenga relación con el hecho de la informalidad en la mayoria de negocios veterinarios en México donde lo mas probable es que incluso haya mucho mas vetrinarias que ni si quiera estan registradas en este padron, por lo que establecer un hospital que ofrezca los 3 tipos de servicio y que este bien reglamentado pareciera ser un buen diferenciador.

In [10]:
# Agrupar por estado
conteo_estados = df['estado'].value_counts().reset_index()
conteo_estados.columns = ['estado', 'cantidad']

# Crear gráfico
fig = px.bar(
    conteo_estados,
    x='estado',
    y='cantidad',
    text='cantidad',
    title='Distribución de empresas veterinarias por estado',
    labels={'estado': 'Estado', 'cantidad': 'Cantidad de empresas'},
    template='plotly_white',
    color='estado'
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Observamos que la capital del país es por mucho el lugar con mas veterinarias registradas, esto en gran medida se debe tanto a la concentración de la población en la zona y a las regulaciones mas estricas que hay en la capital.
La opción quiza mas viable para un nuevo hospital seria buscar un estado con gran crecimiento donde no haya un gran numero de negocios establecidos, por lo que Queretaro me pareceria una buena opción.